## This notebook is devoted for arbitrage opportunity from fat finger event

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import yfinance as yf
from sklearn.cluster import KMeans
from colorama import Fore, Style
import seaborn as sns
from sklearn.metrics import r2_score
sns.set(style='darkgrid')
import plotly.graph_objects as go
import pickle

import sys
sys.path.append('../src') 
from stock_strategy import stock_strategy, stock_regression
from chinese_index_etf_strategy import etf_strategy, etf_regression

from scipy.stats import norm
from scipy.optimize import minimize
from scipy.optimize import fsolve

from kneed import KneeLocator

### Near ATM and long-dated option?

Validated using call-put parity, these "fat finger error" carries much less profitability than originally thought.

In [2]:
def option_symbol(
    strike: float,
    # expiration: datetime = datetime.today() + relativedelta(days=(4 - datetime.today().weekday()) % 7) + relativedelta(weeks=8), 
    # expiration: datetime = pd.Timestamp(year=(datetime.today() + relativedelta(months=5)).year, month=(datetime.today() + relativedelta(months=5)).month, day=1) + pd.offsets.MonthEnd(0) - pd.offsets.Week(weekday=4), 
    expiration: datetime = pd.Timestamp(year=(datetime.today()+relativedelta(months=5)).year, month=(datetime.today()+relativedelta(months=5)).month, day=1) + pd.offsets.Week(weekday=4) + pd.offsets.Week(2), 
    stock_name: str = 'QQQ',
    option_type: str = 'C',
):
    '''
    Return the symbol of an option, given the strike price and the expiration
    '''
    leading_zeros = 4 - int(np.log10(strike))
    ending_zeros = 8 - leading_zeros - len(str(strike*1000))
    return stock_name.upper() + expiration.strftime('%Y%m%d')[2:] + option_type + '0'*leading_zeros + str(strike*1000) + '0'*ending_zeros

In [3]:
df_qqq = yf.download('qqq'.upper(),
                     start=(datetime.today() - relativedelta(months=4)).strftime('%Y-%m-%d'),
                     prepost=True,
                     auto_adjust=True
                     ).droplevel(level='Ticker', axis=1)
df_qqq.index = pd.to_datetime(df_qqq.index)
df_qqq.columns = df_qqq.columns.str.lower()
print(df_qqq.shape)
df_qqq.head()

[*********************100%***********************]  1 of 1 completed

(87, 5)


Price,close,high,low,open,volume
Date,,,,,
2025-08-05,559.621277,565.903972,559.081863,564.735343,48666600
2025-08-06,566.663086,567.102579,559.980830,560.460255,41823700
2025-08-07,568.580872,572.656164,564.455649,571.008051,44463000
2025-08-08,573.884705,574.104482,569.489836,569.789476,35255500
2025-08-11,572.186646,576.132084,570.908159,574.024542,33112900


In [5]:
days_till_expirations = []
call_option_symbols = []
option_open = []
option_high = []
option_low = []
option_close = []
option_volume = []
# put_option_symbols = []
# put_option_open = []
# put_option_high = []
# put_option_low = []
# put_option_close = []
# put_option_volume = []

# previous_row = None 
for idx, row in df_qqq.iterrows():
    # if previous_row is not None:
        try:
            # strike = int((previous_row['close'] + 2.5) // 5 * 5)
            strike = int((row['open'] + 2.5) // 5 * 5)
            expiration = pd.Timestamp(year=(idx+relativedelta(months=4)).year, month=(idx+relativedelta(months=4)).month, day=1) + pd.offsets.Week(weekday=4) + pd.offsets.Week(2)
            call_option_symbol = option_symbol(
                strike=strike,
                expiration=expiration,
            )
            df_option = yf.download(call_option_symbol,
                    start=idx.strftime('%Y-%m-%d'),
                    prepost=True,
                    auto_adjust=True
                    ).droplevel(level='Ticker', axis=1)
            # put_option_symbol = option_symbol(
            #     strike=strike,
            #     expiration=expiration,
            #     option_type='P',
            # )
            # df_put_option = yf.download(put_option_symbol,
            #         start=idx.strftime('%Y-%m-%d'),
            #         prepost=True,
            #         auto_adjust=True
            #         ).droplevel(level='Ticker', axis=1)
            # if df_put_option[df_put_option.index==idx].shape[0] > 0 and df_option[df_option.index==idx].shape[0] > 0:
            option_open.append(df_option[df_option.index==idx]['Open'][0])
            option_high.append(df_option[df_option.index==idx]['High'][0])
            option_low.append(df_option[df_option.index==idx]['Low'][0])
            option_close.append(df_option[df_option.index==idx]['Close'][0])
            option_volume.append(df_option[df_option.index==idx]['Volume'][0])
            call_option_symbols.append(call_option_symbol)
            days_till_expirations.append((
                pd.Timestamp(year=(idx+relativedelta(months=4)).year, month=(idx+relativedelta(months=4)).month, day=1)
                +
                pd.offsets.Week(weekday=4)
                +
                pd.offsets.Week(2) - idx).days)
            # put_option_open.append(df_put_option[df_put_option.index==idx]['Open'][0])
            # put_option_high.append(df_put_option[df_put_option.index==idx]['High'][0])
            # put_option_low.append(df_put_option[df_put_option.index==idx]['Low'][0])
            # put_option_close.append(df_put_option[df_put_option.index==idx]['Close'][0])
            # put_option_volume.append(df_put_option[df_put_option.index==idx]['Volume'][0])
            # put_option_symbols.append(put_option_symbol)
            # else:
            #     df_qqq.drop(idx, inplace=True)
        except:
            df_qqq.drop(idx, inplace=True)
        # previous_row = row
    
df_qqq['call_option_symbols'] = call_option_symbols
df_qqq['option_open'] = option_open
df_qqq['option_high'] = option_high
df_qqq['option_low'] = option_low
df_qqq['option_close'] = option_close
df_qqq['option_volume'] = option_volume

# df_qqq['put_option_symbols'] = put_option_symbols
# df_qqq['put_option_open'] = put_option_open
# df_qqq['put_option_high'] = put_option_high
# df_qqq['put_option_low'] = put_option_low
# df_qqq['put_option_close'] = put_option_close
# df_qqq['put_option_volume'] = put_option_volume
# df_qqq['days_till_expiration'] = days_till_expirations

print(df_qqq.shape)
df_qqq

[*********************100%***********************]  1 of 1 completed
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_open.append(df_option[df_option.index==idx]['Open'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_high.append(df_option[df_option.index==idx]['High'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future ver

(66, 11)



/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:42: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_open.append(df_option[df_option.index==idx]['Open'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  option_high.append(df_option[df_option.index==idx]['High'][0])
/var/folders/nb/2_t_pyss0gs6mwvk974rxmz80000gn/T/ipykernel_9143/3670820244.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with

Price,close,high,low,open,volume,call_option_symbols,option_open,option_high,option_low,option_close,option_volume
Date,,,,,,,,,,,
2025-08-05,559.621277,565.903972,559.081863,564.735343,48666600,QQQ251219C00565000,29.950001,29.950001,28.799999,29.440001,47
2025-08-06,566.663086,567.102579,559.980830,560.460255,41823700,QQQ251219C00560000,33.169998,35.419998,33.070000,35.419998,24
2025-08-07,568.580872,572.656164,564.455649,571.008051,44463000,QQQ251219C00570000,31.940001,32.529999,27.950001,30.510000,0
2025-08-08,573.884705,574.104482,569.489836,569.789476,35255500,QQQ251219C00570000,31.780001,33.169998,31.780001,33.169998,106
2025-08-11,572.186646,576.132084,570.908159,574.024542,33112900,QQQ251219C00575000,29.910000,30.879999,28.000000,28.000000,85
...,...,...,...,...,...,...,...,...,...,...,...
2025-12-01,617.169983,619.440002,612.520020,613.630005,40957600,QQQ260417C00615000,35.750000,38.340000,35.750000,37.430000,61
2025-12-02,622.000000,623.750000,617.590027,619.460022,54541800,QQQ260417C00620000,37.130001,37.759998,35.869999,37.250000,251
2025-12-03,623.520020,624.219971,618.030029,619.619995,47841600,QQQ260417C00620000,36.000000,38.130001,36.000000,37.990002,28


In [ ]:
# # Don't do arbitrage from put-call parity violation -- too complicated
# df_qqq['risk_free_rate_from_parity'] = 365/df_qqq['days_till_expiration']*np.log(
#     ((df_qqq['open'] + 2.5) // 5 * 5).astype(int)
#     /
#     (df_qqq['put_option_close']+df_qqq['close']-df_qqq['option_close'])
# )
# df_qqq